In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

import shap

Load sick and healthy patients

In [2]:
with open("data/eid_sick") as f:
    eid_sick = list(map(int, f.read().split('\n')))

with open("data/eid_healthy") as f:
    eid_healthy = list(map(int, f.read().split('\n')))

len(eid_sick), len(eid_healthy)

(26314, 476056)

Load column descriptions

In [3]:
column_desc = {}

with open("data/my_columns") as f:
    my_columns = f.read().split('\n')

with open("data/filtered_fields") as f:
    lines = f.read().split('\n')
    category = ''
    
    for line in lines:
        if line == '':
            continue
        
        # New category
        if line[0] == '#':
            category = line.split(' - ')[-1]
            column_desc[category] = {}
            
        # Fill category
        else:
            field, fdesc, ftype = line.split('\t')
            column_desc[category][f'{field}-0.0'] = (fdesc, ftype)
    
print(f'{len(column_desc)} categories, total of {sum([len(fields) for fields in column_desc.values()])} columns')
print('===================================')
for category, cols in column_desc.items():
    print(f'{category}: {len(cols)} columns')

11 categories, total of 193 columns
Physical activity: 24 columns
Sleep: 7 columns
Smoking: 26 columns
Diet: 28 columns
Alcohol: 19 columns
Chest pain: 4 columns
Blood pressure: 8 columns
Arterial stiffness: 8 columns
Body size measures: 8 columns
Blood count: 31 columns
Blood biochemistry: 30 columns


In [4]:
filtered_columns = [
    col
    for cols in column_desc.values()
    for col in cols
]

Read my data

In [5]:
df_sick = pd.read_csv('data/sick_data.csv', 
                      usecols=['eid'] + filtered_columns, 
                      index_col=['eid'])
df_healthy = pd.read_csv('data/my_data.csv', 
                         usecols=['eid'] + filtered_columns, 
                         index_col=['eid'],
                         nrows=2 * len(df_sick))
df_healthy = df_healthy.loc[list(set(eid_healthy) & set(df_healthy.index))[:len(df_sick)]]

len(df_sick), len(df_healthy)

(26314, 26314)

In [6]:
df_sick['diagnosis'] = 1
df_healthy['diagnosis'] = 0
df_raw = pd.concat([df_sick, df_healthy])

In [7]:
eid_train, eid_test = train_test_split(df_raw.index.to_list(), test_size=0.1, random_state=42)
eid_train, eid_val = train_test_split(eid_train, test_size=0.1, random_state=42)

len(eid_train), len(eid_val), len(eid_test)

(42628, 4737, 5263)

### Impute missing & negative values

In [8]:
column_missing_values = pd.Series()

for category, cols in column_desc.items():
    print()
    print(category)
    nulls = df_raw[cols.keys()].isna().sum(axis=0)
    negs = (np.logical_and(df_raw[cols.keys()] < 0, df_raw[cols.keys()] > -10)).sum(axis=0)
    print(negs + nulls)
    
    column_missing_values = pd.concat([column_missing_values, nulls + negs])


Physical activity
1100-0.0     1868
2634-0.0    32389
1021-0.0    28641
894-0.0     14801
3647-0.0    30357
1001-0.0    48309
914-0.0     25935
874-0.0      8400
981-0.0     17433
2624-0.0    32453
1011-0.0    28523
3637-0.0    30376
943-0.0      1247
991-0.0     48311
971-0.0     17427
884-0.0      3159
904-0.0      3253
864-0.0      1397
1090-0.0     1172
1080-0.0     1126
1070-0.0      628
6164-0.0     5093
6162-0.0     1071
924-0.0       658
dtype: int64

Sleep
1160-0.0     499
1170-0.0     677
1180-0.0    6445
1190-0.0     212
1200-0.0     185
1210-0.0    3897
1220-0.0     472
dtype: int64

Smoking
20160-0.0      376
20162-0.0    33819
20161-0.0    33819
20116-0.0      389
1239-0.0       163
1249-0.0      4969
2644-0.0     40178
3436-0.0     48079
3456-0.0     48410
3466-0.0     48451
3476-0.0     48436
3486-0.0     48057
3496-0.0     48160
3506-0.0     48050
6158-0.0     51109
2867-0.0     36981
2877-0.0     36927
2887-0.0     38009
2897-0.0     36998
2907-0.0     37278
6157-0.0

In [9]:
for missing_th in range(5000, 50001, 5000):
    print(f'taking only columns with less than {missing_th} missing values, we are left with {len(column_missing_values[column_missing_values < missing_th])} columns')

taking only columns with less than 5000 missing values, we are left with 107 columns
taking only columns with less than 10000 missing values, we are left with 125 columns
taking only columns with less than 15000 missing values, we are left with 129 columns
taking only columns with less than 20000 missing values, we are left with 136 columns
taking only columns with less than 25000 missing values, we are left with 137 columns
taking only columns with less than 30000 missing values, we are left with 141 columns
taking only columns with less than 35000 missing values, we are left with 156 columns
taking only columns with less than 40000 missing values, we are left with 164 columns
taking only columns with less than 45000 missing values, we are left with 168 columns
taking only columns with less than 50000 missing values, we are left with 190 columns


In [10]:
df_raw[df_raw == -10] = 0
df_raw[df_raw < 0] = np.nan

In [34]:
common_columns = column_missing_values[column_missing_values < len(df_raw) // 5].index.to_list() + ['diagnosis']
category_classifiers = {}

for category, cols in column_desc.items():
    c = list(set(cols.keys()) & set(common_columns)) + ['diagnosis']
    
    if len(c) > 1:
    
        df_train = df_raw[c].loc[eid_train].dropna()
        df_val = df_raw[c].loc[eid_val].dropna()
        
        X_train = df_train.drop(columns=['diagnosis'])
        y_train = df_train['diagnosis']
        X_val = df_val.drop(columns=['diagnosis'])
        y_val = df_val['diagnosis']
        
        clf = LogisticRegression(max_iter=2000).fit(X_train, y_train)
        category_classifiers[category] = clf
        
        print(category)
        print(F'\ttrain score: {100 * clf.score(X_train, y_train):.3f} f1: {f1_score(y_train, clf.predict(X_train)):.3f}, auc: {roc_auc_score(y_train, clf.predict(X_train)):.3f}') 
        print(f'\tval score: {100 * clf.score(X_val, y_val):.3f}, f1: {f1_score(y_val, clf.predict(X_val)):.3f}, auc: {roc_auc_score(y_val, clf.predict(X_val)):.3f}')

Physical activity
	train score: 59.376 f1: 0.555, auc: 0.591
	val score: 59.273, f1: 0.559, auc: 0.591
Sleep
	train score: 58.178 f1: 0.553, auc: 0.581
	val score: 58.423, f1: 0.556, auc: 0.584
Smoking
	train score: 58.849 f1: 0.503, auc: 0.585
	val score: 58.141, f1: 0.497, auc: 0.579
Diet
	train score: 58.488 f1: 0.572, auc: 0.585
	val score: 58.014, f1: 0.569, auc: 0.580
Alcohol
	train score: 55.078 f1: 0.543, auc: 0.551
	val score: 53.684, f1: 0.533, auc: 0.537
Chest pain
	train score: 61.298 f1: 0.490, auc: 0.612
	val score: 61.468, f1: 0.491, auc: 0.615
Blood pressure
	train score: 58.640 f1: 0.573, auc: 0.586
	val score: 59.815, f1: 0.584, auc: 0.598
Body size measures
	train score: 64.818 f1: 0.653, auc: 0.648
	val score: 64.307, f1: 0.652, auc: 0.643
Blood count
	train score: 63.090 f1: 0.625, auc: 0.631
	val score: 63.091, f1: 0.627, auc: 0.631
Blood biochemistry
	train score: 69.987 f1: 0.719, auc: 0.698
	val score: 70.785, f1: 0.731, auc: 0.705


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
# new_sick = []
# new_healthy = []

# for i in range(len(df_raw)):
#     new_features = []
    
#     for category, clf in category_classifiers.items():
#         cols = column_desc[category].keys()
#         c = list(set(cols) & set(common_columns))
        
#         feature = clf.predict_proba(df_raw[cols].iloc[i].to_numpy().reshape(1, -1)) if df_raw[cols].iloc[i].isna().sum() == 0 else np.nan
#         new_features.append(feature)
        
#     if df_raw.iloc[i]['eid'] in eid_sick:
#         new_sick.append(new_features)
#     else:
#         new_healthy.append(new_features)